In [2]:
import logging

In [3]:
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [4]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()
print(os.getenv("GROQ_MODEL"))

llama-3.3-70b-versatile


In [5]:

try:
    router_llm = ChatGroq(
        model=os.getenv("GROQ_MODEL"),
        api_key=os.getenv("GROQ_API_KEY")
    )
    logger.info("Successfully connected to the router LLM")
except Exception as e:
    logger.exception("Failed to connect to the router LLM")


2025-10-15 13:09:06,632 - __main__ - INFO - Successfully connected to the router LLM


In [6]:
router_llm.invoke("Give me a financial tip")

2025-10-15 13:09:07,909 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Here\'s a financial tip:\n\n**Pay Yourself First**\n\nAs soon as you receive your paycheck, set aside a portion of it into a separate savings or investment account. This is called "paying yourself first." By doing so, you\'ll ensure that you\'re prioritizing your own financial goals, such as building an emergency fund, paying off debt, or saving for retirement.\n\nTry to allocate at least 10% to 20% of your income towards savings and investments. This habit will help you develop a disciplined approach to managing your finances and make progress towards your long-term financial goals.\n\nRemember, it\'s essential to prioritize your own financial well-being, just as you would any other important bill or expense. By paying yourself first, you\'ll be taking a significant step towards achieving financial stability and security.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 162, 'prompt_tokens': 40, 'total_tokens': 202, 'completion_time': 